In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import os
import sys

from collections import defaultdict
from sklearn.mixture import GaussianMixture

sys.path.append(os.getcwd()[:-10])
from Filtering.filtering import *

### In this notebook, we will be clustering securities in our trading universe.

As an example, we will download 6 months of historical stock data for our trading universe, preprocess the data to obtain observation values and perform clustering using a Gaussian Mixture Model.

More precisely, the observations are smoothed daily returns obtained using the formulas provided by Zura Kakushadze and Willie Yu
    \begin{align*}
    \\
    &S^i_t := \text{Security } i \text{ close price, time } t\\
    &R^i_t := \text{Returns of } S^i_t\\
    &\sigma^i := \text {Serial standard deviation of }R^i_t\\
    &\tilde{R^i_t} := \frac{R^i_t}{\sigma^i}, \quad \text{"Normalised Returns"}\\
    &\hat{R^i_i} := \frac{\tilde{R^i_t}}{u_i}, \quad \text{"Smoothed Returns"}\\
    &u_i := \max(
                 \exp(
                      \log(\sigma_i) - (
                          \text{Median}(\log(\sigma_i)) - 3 * \text{Mean Absolute Deviation}(\log(\sigma_i))
                                       )
                      )
    , 1)\\\\
    &\text{Median(·) and MAD(·) above are cross-sectional.}
    \end{align*}

References:
- Z. Kakushadze, W. Yu. Statistical Industry Classification. arXiv:1607.04883


### Filtering Trading Universe

In [2]:
listed_companies = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0].set_index('Symbol')

# pd.to_datetime cannot be called directly due to inconsistent data structures
def try_mapping_to_datetime(date):
    try:
        return pd.Timestamp(date)
    except:
        return np.nan
    
# Removing companies listed after 2010/01/01
listed_companies['Date first added'] = listed_companies['Date first added'].map(try_mapping_to_datetime)
listed_companies = listed_companies.dropna()
listed_companies = listed_companies[listed_companies['Date first added'] < pd.Timestamp('2010-01-01')]

# Filtering
trading_universe = filter_universe(securities = listed_companies.index.to_list(),
                                   current_time = pd.Timestamp('2010-01-01'),
                                   lookback=30,
                                   percentile=.1)

print(trading_universe)

[*********************100%***********************]  256 of 256 completed
['GOOG', 'AMZN', 'AAPL', 'BAC', 'C', 'CMCSA', 'GS', 'MSFT', 'ORCL', 'UNH', 'V']


In [3]:
daily_bar_data = yf.download(tickers=trading_universe,
                             start='2009-07-01',
                             end='2010-01-01')['Adj Close']

daily_bar_data = daily_bar_data.dropna(axis=1)
daily_bar_data.head()

[*********************100%***********************]  11 of 11 completed


,AAPL,AMZN,BAC,C,CMCSA,GOOG,GS,MSFT,ORCL,UNH,V
Date,,,,,,,,,,,
2019-07-01,49.314014,1922.189941,27.774740,65.024223,40.442707,1097.949951,196.166092,132.249786,55.461327,233.399399,171.285980
2019-07-02,49.602726,1934.310059,27.519838,64.748459,40.813831,1111.250000,195.331589,133.127045,55.891560,234.678757,172.605530
2019-07-03,50.013779,1939.000000,27.425430,65.125328,41.223022,1121.579956,195.388474,133.984802,56.273987,235.871552,174.171280
2019-07-05,49.969738,1942.910034,27.623688,65.630882,41.184952,1131.589966,197.152283,133.594910,56.675526,237.583755,173.964478
2019-07-08,48.939671,1952.319946,27.567043,65.382706,40.566422,1116.349976,195.113464,133.497437,56.914539,238.401352,173.501694


### Creating observation features

In [4]:
# calculate returns
returns = daily_bar_data.pct_change()
returns = returns.dropna()

# normalising returns
standard_deviation = returns.std(axis=0)
normalised_returns = returns / standard_deviation

# smoothing_returns
log_standard_deviation = np.log(standard_deviation)
smoothing_factor = log_standard_deviation - (log_standard_deviation.median() - 3 * log_standard_deviation.mad())
smoothing_factor = np.exp(smoothing_factor)
smoothing_factor[smoothing_factor < 1] = 1
smoothed_returns = normalised_returns / smoothing_factor

In [5]:
smoothed_returns.head()

,AAPL,AMZN,BAC,C,CMCSA,GOOG,GS,MSFT,ORCL,UNH,V
Date,,,,,,,,,,,
2019-07-02,0.292820,0.475828,-0.478822,-0.193002,0.646718,0.592358,-0.227474,0.543693,0.523850,0.231287,0.552022
2019-07-03,0.414475,0.182971,-0.178984,0.264886,0.706570,0.454569,0.015572,0.528104,0.462057,0.214462,0.650010
2019-07-05,-0.044042,0.152175,0.377161,0.353277,-0.065086,0.436433,0.482704,-0.238512,0.481851,0.306294,-0.085081
2019-07-08,-1.031015,0.365488,-0.106986,-0.172088,-1.058421,-0.658581,-0.552974,-0.059802,0.284787,0.145205,-0.190621
2019-07-09,0.305052,1.390757,0.268011,0.275109,0.578607,0.371457,0.524968,-0.299229,0.079412,-0.313252,0.626291


### Clustering

In [6]:
securities = smoothed_returns.columns
clustering_model = GaussianMixture(n_components=5).fit(smoothed_returns.values.T)
cluster_tags = clustering_model.predict(smoothed_returns.values.T)

clusters_of_securities = defaultdict(list)
for i in range(smoothed_returns.shape[1]):
    clusters_of_securities[cluster_tags[i]].append(securities[i])

In [7]:
clusters_of_securities

defaultdict(list,
            {3: ['AAPL', 'GOOG'],
             0: ['AMZN', 'MSFT', 'V'],
             1: ['BAC', 'C', 'GS', 'ORCL'],
             2: ['CMCSA'],
             4: ['UNH']})

### Improvements

Clustering is a complex task. The algorithm presented here is overly simplistic and hence there are many areas of improvements. Some questions to consider are:

- Optimal clustering length: Securities rarely exhibit highly correlated behaviours over long periods of time (6 months in this case). On the other hand, choosing a period that is too short leads to spurious results.
- Number of clusters: Getting this number algorithmically rather than discretionally setting it apriori
- Randomness of clustering algorithms: How to ensure consistent performance
- Clustering algorithm: Hierarchical clustering VS Gaussian Mixture Models
- ONC algorithm suggested by López de Prado